# Datová akademie, ČSOB, 2023

---

* [Časové řady](#Časové-řady),
    - [úvodní motivace](#Úvod),
    - [základní objekty](#Základní-objekty),
    - [indexování časovými údaji](#Indexování-s-časem),
    - [funkce to_datetime](#Funkce-TO_DATETIME),
    - [funkce date_range](#Funkce-DATE_RANGE),
    - [resampling](#Resampling,-snížení-frekvence),
    - [zpřesňování](#Zpřesňování-řady),
    - [cvičení](#🧠-CVIČENÍ-🧠,-procvič-si-časové-řady).
* [Výkonné pandy](#Výkonný-PANDAS),
    - [funkce eval](#Funkce-EVAL),
    - [výhody eval](#Výkon-EVAL),
    - [funkce query](#Funkce-QUERY),
    - [souhrn k eval a query](#Souhrn-k-EVAL-a-QUERY),
    - [cvičení](#🧠-CVIČENÍ-🧠,-procvič-si-funkce-EVAL-a-QUERY).
* [Skutečné hodnoty](#Skutečné-datové-sety),
    - [chybějící údaje](#Kolik-mi-chybí-údajů),
    - [encoding](#Encoding),
    - [nekonzistentní data](#Nekonzistentní-data),
    - [cvičení](#🧠-CVIČENÍ-🧠,-procvič-si-práci-s-chybným-datasetem).

<br>

## Časové řady

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.odqbr-09dxxBQjF7x7NyBAHaII%26pid%3DApi&f=1&ipt=5b4a03c1dd38b52108237102ebb350595a5d6ea6a50572f5897ca89fbf5b5445&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

### Úvod

---

Část *frameworku* byla vyvinuta za účelem **finančního modelování**.

Proto je více než dobře vybavena sadou nástrojů, které umí pracovat **s daty, časem a časovými objekty**.

<br>

Uplatnění:
* **Analýza a predikce**: umožňuje analyzovat vývoj hodnot v čase a předpovídat budoucí trendy nebo sezónní změny,
* **Efektivní manipulace s časem**: zjednodušuje úkoly, jako je agregace, interpolace nebo časové posuny,
* **Flexibilní indexace**: podporuje flexibilní indexaci časových řad, což usnadňuje filtrování, řazení a výběr dat,
* **Kompatibilita s dalšími knihovnami**: je kompatibilní s mnoha dalšími knihovnami pro analýzu časových řad, jako je `statsmodels` nebo `scikit-learn`.

<br>

#### Pomocí standardního Pythonu

---

Standardní výbavou Pythonu jsou knihovny `datetime`:

In [ ]:
from datetime import datetime

In [ ]:
# help(datetime)

In [ ]:
datetime(year=2023, month=4, day=5)

In [ ]:
type(datetime(year=2023, month=4, day=5))

<br>

Nebo knihovna `dateutil` pro *parsování* datových typů z různých **stringových zadání**:

In [ ]:
from dateutil import parser

In [ ]:
type(parser)

<br>

Funkce `parse` zpracuje (*parsuje*) velké množství různých formátů času ve `str`:

In [ ]:
date = parser.parse("5th of april, 2023")

In [ ]:
type(parser.parse)

In [ ]:
date.year

In [ ]:
date.month

In [ ]:
date.day

<br>

Kde pomocí metody `strftime` můžeš vypsat den:

In [ ]:
date.strftime("%A")

In [ ]:
date.strftime("%d/%m/%y")

<br>

#### Pomocí NUMPY

---

Některé nedostatky uvnitř knihoven `datetime` a `dateutil` vedli ke vzniku sady nástrojů.

Tyto doplňky vznikly pod hlavičkou knihovny `numpy`.

In [ ]:
import numpy
from numpy import array, arange

In [ ]:
date = array('2023-04-05', dtype=numpy.datetime64)

In [ ]:
date

<br>

Pokud potřebuješ pole **následujících 7 dní**:

In [ ]:
date + arange(7)

<br>

Vzhledem k jednotnému datovu typu v poli pro **numpy** `datetime64` může tento typ operace
provádět mnohem rychleji.

Přímo v Pythonu pomocí `datetime` objektu potom roste handicap, když objekty nabývají na velikosti.

<br>

#### Pomocí PANDAS

---

Jde o kombinace objektů z obou předchozích podkapitol.

Ty dávají dohromady to nejlepší prostředky pro zacházení s časem.

In [ ]:
from pandas import to_datetime

In [ ]:
date = to_datetime("5th of April, 2023")

In [ ]:
date

In [ ]:
type(date)

In [ ]:
date.strftime("%A")

<br>

### Základní objekty

---

V podstatě jde o hlavní nástroj, který tato knihovna dovede nabídnout.

Tedy práce **s univerzálními datovými typy** pro čas.

Mezi základní objekty pro práci s časem patří:
* `Timestamp`, typ (související struktura Indexu `DatetimeIndex`),
* `Period`, typ (.. `PeriodIndex`),
* `Timedelta`, typ (.. `TimedeltaIndex`).

<br>

#### Timestamp

---

Základní datový typ do časových řad.

Přiřazuje pandasí hodnoty k skutečnému konkrétním časovému okamžiku.

In [ ]:
from pandas import Timestamp

In [ ]:
# Timestamp?

In [ ]:
Timestamp("2023-05-04")  # str, tuple, datetime.datetime

In [ ]:
Timestamp(2023, 5, 4)

In [ ]:
type(Timestamp(2023, 5, 4))

<br>

#### Period

---

V mnoha případech je však přirozenější spojit proměnné, jako je **počáteční čas** s následným **časovým rozpětím**.

`Period` představuje časový interval nebo období, které má začátek a *frekvenci*.

In [ ]:
from pandas import Period

In [ ]:
Period("2023-04", freq="D")

In [ ]:
type(Period("2023-04", freq="D"))

Hlavním rozdílem pro `Timestamp` a `Period` je v rámci jejich zapracování.

`Timestamp` pracuje s konkrétním časovým okamžikem.

`Period` naopak s časovým intervalem.

<br>

#### Timedelta

---

Představuje rozdíl mezi dvěma datumy nebo časy.

In [ ]:
datum_1 = Timestamp(2023,4,12, 12, 30)

In [ ]:
datum_2 = Timestamp(2023,4,10, 12, 0)

In [ ]:
datum_1 - datum_2

In [ ]:
type(datum_1 - datum_2)

In [ ]:
from pandas import Timedelta

In [ ]:
# Timedelta?

<br>

### Indexování s časem

---

Pomocí objektů času můžeš vytvořit samotné **Indexy**.

`DatetimeIndex` obsahuje časové značky (*timestamp*), které jsou uloženy ve formátu `datetime64` s nanosekundovou přesností.

Tento objekt umožňuje efektivní práci s časovými řadami a poskytuje mnoho funkcí pro manipulaci s daty a časy:

In [ ]:
from pandas import DatetimeIndex, Series

<br>

Pomocný objekt typu `list`:

In [ ]:
datumy = ["2023-04-05", "2022-04-05", "2021-04-05", "2020-04-05"]

In [ ]:
indexy = DatetimeIndex(datumy)

In [ ]:
type(indexy)

In [ ]:
jmena_dnu = [to_datetime(den).strftime("%A") for den in datumy]

In [ ]:
jmena_dnu

In [ ]:
hodnoty_sl = Series(jmena_dnu, index=indexy)

In [ ]:
hodnoty_sl

In [ ]:
type(hodnoty_sl)

In [ ]:
type(indexy)

In [ ]:
hodnoty_sl["2020": "2022"]

In [ ]:
hodnoty_sl["2020-04-05": "2022-04-05"]

<br>

#### TimedeltaIndex

---

Jde o rozdíl v jednotkách času.

Nejčastěji se s tímto objektem setkáš, pokud potřebuješ získat **rozdíl mezi dvěma datumy**.

In [ ]:
datumy

In [ ]:
datumy_ = to_datetime(datumy)

In [ ]:
datumy_

In [ ]:
specialni_datumy = to_datetime([
    datetime(2022, 4, 5), "5th of April 2021", "2020-Apr-5", "05-04-2019"]
)

In [ ]:
datumy_ - specialni_datumy

<br>

### Funkce TO_DATETIME

---

Nejčastější datové typy, které lze zapsat přímo, ovšem běžnější je pracovat s funkcí `to_datetime`.

Funkce `to_datetime` umí parsovat různé stringové formáty.

In [ ]:
from pandas import to_datetime

<br>

#### Práce s jedním datumem

---

In [ ]:
datum = to_datetime("05/04/2023")  # str, tuple, datetime.datetime

In [ ]:
datum

In [ ]:
type(datum)

Pokud do funkce `to_datetime` vložíš jedinou hodnotu, vrací objekty typu `Timestamp`.

<br>

#### Specifický formát

---

Pokud se parser ve funkci `to_datetime` ztratí nebo tvoje zadání neodpovídá jeho vyhotovení, můžeš zadat formát explicitně:

In [ ]:
vystup = to_datetime("12-11-2010 01:05")

In [ ]:
vystup.day

<br>

V ukázce výš ale hodnota `11` představuje měsíc, nikoliv den.

In [ ]:
vystup = to_datetime("12-11-2010 01:05", format="%d-%m-%Y %H:%M")

In [ ]:
vystup.day

In [ ]:
vystup.month

In [ ]:
vystup.year

In [ ]:
vystup.hour

<br>

#### Časové zóny

---

Objekt `Timestamp` umožňuje hodnoty časových zón zadávat pomocí metody `tz_localize`:

In [ ]:
datum

In [ ]:
datum.tz

In [ ]:
tz_datum = datum.tz_localize("Europe/Prague")

In [ ]:
tz_datum.tz

In [ ]:
nove_tz_datum = tz_datum.tz_convert("US/Pacific")

In [ ]:
nove_tz_datum = tz_datum.tz_convert("Europe/London")

In [ ]:
nove_tz_datum

<br>

#### Práce s několika datumy

---

Stejně jak umí funkce *parsovat* jednu hodnotu, umí zpracovat pole časových stringů:

In [ ]:
datumy = to_datetime(
    [datetime(2023, 4, 5), "5th of April 2023", "2023-Apr-5", "04-05-2023", "20230405"]
)

In [ ]:
datumy

In [ ]:
type(datumy)

Zatímco pole hodnot s datumy, které funkce `to_datetime` zpracuje, vrací objekt typu `DatetimeIndex`.

<br>

Objekt `DatetimeIndex` potom můžeš konvertovat na `PeriodIndex` pomocí *metody* `to_period`:

In [ ]:
datumy.to_period?

<br>

Tato metoda je užitečná, pokud chcete převést časové řady **na určité časové období**.

Když budeš třeba potřebovat převést hodnoty **z denních dat na měsíční data**.

#### Denní data

In [ ]:
datumy.to_period("D")

#### Měsíční data

In [ ]:
datumy.to_period("M")

<br>

### Funkce DATE_RANGE

---

In [ ]:
from pandas import date_range, DataFrame

In [ ]:
date_range(start='2023-01-01', periods=10, freq='D')

In [ ]:
date_range(start='2023-01-01', periods=10, freq='M')

In [ ]:
data = {
    'date': date_range(start='2022-01-01', periods=10, freq='D'),
    'sales': [100, 102, 105, 107, 110, 112, 115, 117, 120, 122]
}

In [ ]:
data

In [ ]:
df_data = DataFrame(data)

In [ ]:
df_data.head(3)

In [ ]:
df_data.set_index('date', inplace=True)

In [ ]:
df_data.head(3)

In [ ]:
df_period_y = df_data.to_period(freq='M')

In [ ]:
df_period_y

<br>

#### Počet period

---

Nachystá objekt, který vypracuje tolik period, které definuješ pomocí parametru `periods`:

In [ ]:
date_range(start='2000-01-01', periods=10, freq='Y')

In [ ]:
# date_range?

In [ ]:
datumy_ind = date_range(start="01-01-1992", periods=8)  # defaultně denní frekvence

In [ ]:
datumy_ind

<br>

#### Frekvence

---

In [ ]:
datumy_mesicne_ind = date_range("01-01-1992", periods=8, freq="M")

In [ ]:
datumy_mesicne_ind

<br>

### Frekvence

Časovou paletou frekvencí, kterou framework `pandas` nabízí je tato tabulka:

| String | Popisek |
| :-: | :- |
| `D` | kalendářní den |
| `W` | týden |
| `M` | konec měsíce |
| `Q` | konec čtvrtletí |
| `A` | konec roku |
| `H` | hodiny |
| `T` | minuty |
| `S` | vteřiny |
| `B` | pracovní den |
| `BM` | konec pracovníḧo měsíce |
| `BQ` | konec pracovního čtvrtletí |

<br>

#### Hodinové periody

---

In [ ]:
from pandas import timedelta_range

In [ ]:
hodinove_ind = timedelta_range(0, periods=12, freq="H")

In [ ]:
hodinove_ind

#### Netradiční frekvence

---

In [ ]:
timedelta_range(0, periods=10, freq="1H15T")

<br>

### Resampling, snížení frekvence

---

*Resampling* nebo také *převzorkování* je proces, který upravuje frekvenci časové řady.

*Resampling* se obvykle používá **pro snížení frekvence** (z hodinových dat na denní data).

*Resampling* zahrnuje *agregaci* dat.

V `pandas` se k tomu používá metoda `resample`, která má jako parametr novou frekvenci, na kterou chcete data převést:

In [ ]:
data = {
    'date': date_range(start='2022-01-01', periods=60, freq='D'),
    'sales': range(60)
}

In [ ]:
data

In [ ]:
df_data = DataFrame(data)

In [ ]:
df_data

In [ ]:
df_data.set_index('date', inplace=True)

In [ ]:
df_data.head()

In [ ]:
seskupena_data = df_data.resample('Y').sum()

In [ ]:
seskupena_data

<br>

### Zpřesňování řady

---

Zahrnuje **zvýšení frekvence časových řad**.

Přičemž se nově vytvořené hodnoty obvykle interpolují nebo doplňují nějakou konstantou.

V pandas se k zpřesňování používá metoda `asfreq`, která má jako parametr novou frekvenci, na kterou chcete data převést.

In [ ]:
data = {
    'date': date_range(start='2022-01-01', end='2022-03-01', freq='MS'),  # M - konec měsíce
    'sales': [100, 120, 150]
}

In [ ]:
df_data = DataFrame(data)

In [ ]:
df_data.head()

In [ ]:
df_data.set_index('date', inplace=True)

In [ ]:
df_data.head()

In [ ]:
denni_data = df_data.asfreq('D', method='bfill')

In [ ]:
denni_data.head()

Metoda `asfreq` bere jako parametr novou frekvenci, na kterou chceme data převést.

V případě výše jde o frekvenci `D`, což znamená **denní data**.

Protože při zpřesňování se **vytvoří chybějící hodnoty** (v důsledku zvýšení frekvence), je potřeba zvolit metodu pro jejich doplnění.

Výš jde o metodu `forward fill` (parametr `method='ffill'`), která kopíruje předchozí hodnotu pro výplň chybějících hodnot.

Další možností je použít metodu `'back fill'` (parametr `method='bfill'`), která kopíruje následující hodnotu pro výplň chybějících hodnot.

<br>

##### **🧠 CVIČENÍ 🧠, procvič si časové řady**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Celkový prodej za každý měsíc.
2. Průměrný prodej za každý den v týdnu.
3. Denní prodej za poslední týden.

In [ ]:
import pandas as pd
import numpy as np

denni_datumy = pd.date_range(start='2022-01-01', end='2022-12-31', freq='D')
prodeje = np.random.randint(10, 100, size=(len(denni_datumy),))

data = {
    'datumy': denni_datumy,
    'prodeje': prodeje
}

df_prodeje = pd.DataFrame(data)
df_prodeje.set_index('datumy', inplace=True)

In [ ]:
df_prodeje.tail()

In [ ]:
from datetime import timedelta

In [ ]:
#Úkol 1
mesicni_prodej = df_prodeje.resample( 'M' ).sum( )
mesicni_prodej['mesic'] = mesicni_prodej.index.month_name( )
mesicni_prodej = mesicni_prodej.set_index( 'mesic' )
# display( mesicni_prodej )

In [ ]:
mesicni_prodej

In [ ]:
#Úkol 2
df_prodeje['den'] = df_prodeje.index.day_name( )
prumerne_prodeje = df_prodeje.groupby( by="den" ).mean( )
prumerne_prodeje = prumerne_prodeje

In [ ]:
prumerne_prodeje

In [ ]:
"Timedelta" in dir()

In [ ]:
#Úkol 3
zacatek_posledniho_tydne = df_prodeje.index.max( ) - Timedelta( days = 7 )
posledni_tyden = df_prodeje[df_prodeje.index > zacatek_posledniho_tydne]

In [ ]:
posledni_tyden

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    mesicni_prodej = df_prodeje.resample('M').sum()

    df_prodeje['weekday'] = df_prodeje.index.weekday
    denni_prumer = df_prodeje.groupby('weekday')['prodeje'].mean()

    prodej_posledni_vikend = df_prodeje.loc['2022-12-25':, 'prodeje']
    ```
</details>

## Výkonný PANDAS

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.riOR-y2_yu_2cGdEN2e6-wAAAA%26pid%3DApi&f=1&ipt=638c4516d155ff9768066e17887a621cbad0de6bce639a40be98f5dedbce67ac&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Z předchozích ukázek a popisků (seskupování) tedy můžeš chápat, že skutečný výkon *frameworku* tkví v převedení **základních operací do jazyka C**.

Tyto abstrakce jsou účinné a efektivní pro mnoho běžných případů použití.

Vytvářejí ale spousty dočasných objektů, což může způsobit nepřiměřenou režii výpočetního času a paměti.

<br>

### EVAL a QUERY

---

Přímý přístup k operacím rychlosti C bez nákladné alokace `pandas` poskytuje funkce `eval` a `query`.

Ty spolehájí na funkci `numexpr`.

<br>

### Funkce EVAL

---

Dva důvody, proč používat funkci `eval` místo obyčejného Pythonu:
1. velké objekty typu `DataFrame` jsou vyhodnocovány efektivněji,
2. aritmetické a logické výrazy projdou naráz základním enginem (pomocí `numexpr`).

**Pozor!** Dobrá obecná pomůcka říká, že není nutné využívat funkci `eval` pro tabulky, které jsou kratší než 10 000 řádků. 

### Použití EVAL

---



Funkce `eval` slouží k efektivnímu vyhodnocení **aritmetických výrazů** na:
* objekty `DataFrame`,
* objekty `Series`.

Její největší výhodou je **rychlost a efektivita**, která spočívá v minimalizaci paměťové zátěže a zrychlení operací.

Když pracuješ **s velkými datovými sadami**, paměťová zátěž a rychlost se stávají klíčovými faktory.

Funkce `eval` optimalizuje výpočetní proces tím, že snižuje množství alokované paměti a zrychluje výpočetní operace.

<br>

### Ukázka EVAL

---

Pro zadaný *dummy dataset* vytvoř **nový sloupeček** `sloupec_D`, podle vzoru:
`A + B * C`

In [ ]:
import pandas as pd

In [ ]:
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15]}

In [ ]:
df_data = pd.DataFrame(data)

In [ ]:
df_data.head()

In [ ]:
df_data['sloupec_D'] = df_data['A'] + df_data['B'] * df_data['C']

In [ ]:
df_data.head()

<br>

*Alternativní způsob*, jak provést stejný výpočet, pokud máš dlouhou tabulku nebo sloupeček, je použití funkce `eval`:

In [ ]:
df_data['sloupecek_D'] = df_data.eval('A + B * C')

In [ ]:
df_data.head()

In [ ]:
from pandas import eval as ev

In [ ]:
# ev?

<br>

Funkce `eval` zpracovává zadaný `str` podobně jako built-in funkce `eval`.

Souhrnně lze říci:
* funkci `eval` použij tehdy, pokud je [dataset dlouhý](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html#pandas-eval-performance),
* ve všech ostatních scénářích, kde nepotřebuješ šetřit pamět a čas použij klasickou syntaxi.

### Výkon EVAL

---

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pocet_radku, pocet_sloupcu = 1_000_000, 100

In [ ]:
rng = np.random.RandomState()

In [ ]:
dframe_1, dframe_2, dframe_3, dframe_4 = [DataFrame(rng.rand(pocet_radku, pocet_sloupcu)) for _ in range(4)]

In [ ]:
dframe_1.head()

#### Ukázka sčítání hodnot tabulek

---

In [ ]:
from pandas import DataFrame

In [ ]:
df1 = DataFrame({"A": [1, 2], "B": [3, 4]})

In [ ]:
df2 = DataFrame({"A": [2, 3], "B": [1, 2]})

In [ ]:
df1 + df2

In [ ]:
from pandas import eval as ev

In [ ]:
ev?

In [ ]:
%timeit dframe_1 + dframe_2 + dframe_3 + dframe_4

In [ ]:
%timeit ev("dframe_1 + dframe_2 + dframe_3 + dframe_4", engine="numexpr")  # bez engine="numexpr"

In [ ]:
!pip install numexpr

In [ ]:
# 1.10 s

In [ ]:
# pd.eval?

<br>

### Funkce QUERY

---

Funkce `query` slouží k efektivnímu **filtrování řádků**.

Filtrování probíhá na základě **zadané podmínky**.


Hlavní výhodou jsou:
* **snadná čitelnost**, umožňuje zapisovat podmínky ve snadno čitelné formě, zejména v případě složitějších dotazů,
* **rychlost**, může být rychlejší než tradiční metody filtrování,(zejména pro velké datové sady).

### Ukázka QUERY

---

Pro zadaný *dummy dataset* vyfiltruj takové záznamy, kde:
1. Hodnoty ze sloupce A budou **menší než 4**,
2. hodnoty ze sloupce B budou **větší než 6**,
3. hodnoty ze sloupce C budou **větší než 11**.

In [ ]:
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15],
        'D': ["AA", "BB", "CC", "DD", "EE"]
}

In [ ]:
df_data = DataFrame(data)

In [ ]:
df_data

In [ ]:
df_filtrovane = df_data[(df_data['A'] < 4) & (df_data['B'] > 6) & (df_data['C'] > 11)]  # & - and, | - or

In [ ]:
df_filtrovane

Zapisování pomocí *boolean indexování* je ovšem náročné jak pro zápis, tak pro čtení.

<br>

Pomocí funkce `query`:

In [ ]:
df_filtrovane = df_data.query('A < 4 and B > 6 and C > 11')

In [ ]:
df_filtrovane

#### WHERE 'D' LIKE 'CC'

---

In [ ]:
df_filtrovane_str = df_data.query(
    "A < 4 and B > 6 and C > 11 and D.str.contains('CC')"
)  # LIKE

In [ ]:
df_filtrovane_str

In [ ]:
df_filtrovane_str = df_data.query(
    "A < 4 and B > 6 and C > 11 and ~D.str.contains('CC')"
)  # NOT LIKE

In [ ]:
df_filtrovane_str

#### column_a NOT IN []

---

In [ ]:
"A not in [1, 2]"

In [ ]:
"A < 4 and B > 6 and C > 11 and D in ('AA', 'CC')"

<br>

### Nedostatky QUERY

---

1. **Omezená syntaxe**, má omezenější syntaxi ve srovnání s běžným zápisem v Pythonu a nemusí podporovat [všechny operace](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html#eval-examples),
2. **Názvy sloupců**, pokud názvy sloupců obsahují **mezery nebo znaky**, musíš je uvést pomocí backticků a použít symbol `@` pro reference na proměnné mimo `DataFrame`.

In [ ]:
df_cisla = DataFrame({'Sloupec 1': [1, 2, 3, 4, 5], 'Sloupec 2': [10, 20, 30, 40, 50]})

In [ ]:
df_cisla

In [ ]:
# vystup_1 = df_cisla.query('Sloupec 1 < 3')

In [ ]:
limit = 3

In [ ]:
vystup_2 = df_cisla.query('`Sloupec 1` < @limit')

In [ ]:
vystup_2

### Souhrn k EVAL a QUERY

---

Hlavním účelem těchto funkcí je aplikace na skutečně **velké datové sety**.

Rozdíly u drobnějších datasetů **jsou často minimální**.

Velký vliv na výstupný čas dělá parametr `engine`, který se snaží pracovat s knihovnou `numexpr`.

<br>

#### **🧠 CVIČENÍ 🧠, procvič si funkce EVAL a QUERY**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Vyber studenty s věkem **mezi 20 a 22 lety (včetně)** a email, který obsahuje string **gmail.com**,
2. vypočtěte *BMI* (Body Mass Index) pro každého studenta a přidejte ho do nového sloupce (vzorec: *váha[kg] / (výška[m] * výška[m])*)
3. vyberte studenty s BMI **vyšším než 22** a kteří **pochází z měst Brno a Praha**.

In [ ]:
from pandas import DataFrame

uzivatele = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [19, 20, 21, 22, 23],
    'Height_cm': [160, 170, 175, 180, 165],
    'Weight_kg': [50, 65, 70, 80, 55],
    'Email': ["alice@gmail.com", "bob@tech-users.com", "charlie91@mail.com", "david_bB@gmail.com", "eevee@my_domain.net"],
    'Address': [
        "Vrbová 215/14, 602 00, Brno, Česká republika",
        "Jilmová 862, 400 11, Ústí nad Labem, Česká republika",
        "Kamenická 37/2, 170 00, Praha, Česká republika",
        "Podlipná 1284/6, 460 01, Liberec, Česká republika",
        "Havlíčkova 1703, 500 02, Hradec Králové, Česká republika"
    ]
}

df_uzivatele = DataFrame(uzivatele)

In [ ]:
df_uzivatele

In [ ]:
# Úkol 1
vybrani_uzivatele_1 = df_uzivatele.query( 
    "Age.between( 20, 22 ) and Email.str.contains( 'gmail.com' )" 
)

In [ ]:
# Úkol 2
df_uzivatele['BMI'] = df_uzivatele.eval( 
    'Weight_kg / ( ( Height_cm / 100 ) ** 2 )' 
)

In [ ]:
df_uzivatele

In [ ]:
# Úkol 3
vybrani_uzivatele_2 = df_uzivatele.query( 
    "BMI > 22 and Address.str.split( ', ', expand = True )[2].isin( ['Praha', 'Brno'] )" 
)

In [ ]:
df_uzivatele.query( 
    "BMI > 22")

In [ ]:
df_uzivatele["Address"].str.split(", ", expand=True)[2].isin({"Praha", "Brno"})

In [ ]:
df_uzivatele["Address"].str.split(", ", expand=True)[2].isin({"Praha", "Brno"})

In [ ]:
r1 = r1.rename({"2": "Sloupec_2"})

In [ ]:
r1 = r1[2].rename("rrrr")

In [ ]:
r1.columns

In [ ]:
vybrani_uzivatele_2

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    vyber_uzivatel = df_uzivatele.query("20 <= Age <= 22 and Email.str.contains('gmail.com')")

    df_uzivatele['Height_m'] = df_uzivatele.eval('Height_cm / 100')
    df_uzivatele['BMI'] = df_uzivatele.eval('Weight_kg / (Height_m * Height_m)').round(1)
    
    df_prodeje['weekday'] = df_prodeje.index.weekday
    average_weekday_sales = df_prodeje.groupby('weekday')['prodeje'].mean()
    
    df_uzivatele["City"] = df_uzivatele["Address"].apply(lambda x: x.split(",")[2])
    students_with_high_bmi = df_uzivatele.query('BMI > 22 and City in ["Brno", "Praha"]')
    ```
</details>

<br>

## Skutečné datové sety

---

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.JJu86kKRph1LHt5M7agODQHaHa%26pid%3DApi&f=1&ipt=5cfb9e9f7b451b3a4f7a3ebdc4eb99177d477f20f8145cd818cc4966143f8b2b&ipo=images" width="160" style="margin-left:auto; margin-right:auto"/>

Čištění dat je proces, který patří k samotné práci s daty.

Spolu s některými souvisejícími úkony, patří mezi ty více frustrující.

Nesmyslné datové typy, zkomolené časové údaje, nefungující transformace.

In [ ]:
from pandas import DataFrame

In [ ]:
df_uzivatele = DataFrame({
    'Name': ['Alice', 'Bob', 'Charlie', 'David', 'Eve'],
    'Age': [None, 20, None, 22, 23],
    'Height_cm': [160, 170, 175, None, 165],
    'Weight_kg': [50, 65, 70, 80, 55]
})

In [ ]:
df_uzivatele.head(3)

Ihned po načtení je nejlepší data prozkoumat.

Nejenom datové typy ale také mít povědomí **o chybějících hodnotách**:

In [ ]:
df_uzivatele.info()

<br>

Pomocí metody `info` si uděláš aspoň povrchní přehled o hodnotách, datových typech a chybějících hodnotách.

<br>

### Kolik mi chybí údajů

---

Vždycky přistupuj k datům s mírnou skepsí.

Otázkou potom není, *JESTLI* mi chybí data, ale *KOLIK* dat mi chybí **a kde**.

In [ ]:
chybejici_hodnoty = df_uzivatele.isnull()  # isna()

In [ ]:
chybejici_hodnoty

<br>

Pomocí metody `isnull` nahradíš všechny chybějící hodnoty v tabulce/sloupcích pomocí `True`.

Pokud nechybí, nahradí s `False`.

In [ ]:
type(chybejici_hodnoty)

Takový výstup ale není zcela reprezentativní.

Proto je potřeba, sečíst všechny `True` hodnoty po sloupcích:

In [ ]:
chybejici_hodnoty = df_uzivatele.isnull().sum()

In [ ]:
chybejici_hodnoty

<br>

Opět v jednotkách to nemusí zcela prozrazovat chybovost.

Nejlepším indikátorem ale bývají **procenta z celkového množství hodnot ve sloupci**:

In [ ]:
celkem_zaznamu = df_uzivatele.shape[0]  # 0 ... Indexy, 1 ... sloupce

In [ ]:
celkem_chybi = chybejici_hodnoty

In [ ]:
celkem_chybi

In [ ]:
v_procentech = round(celkem_chybi / celkem_zaznamu * 100, 1)

In [ ]:
v_procentech

<br>

případně celkové množství záznamů ve všech sloupcích:

In [ ]:
vsechny_bunky = df_uzivatele.shape[0] * df_uzivatele.shape[1]  # 5 * 4

In [ ]:
vsechny_chybejici_hodnoty = chybejici_hodnoty.sum()            # 4

In [ ]:
celkem_v_procentech = round(vsechny_chybejici_hodnoty / vsechny_bunky * 100, 1)

In [ ]:
celkem_v_procentech

<br>

V této ukázce tedy chybí 20 % dat a to není málo!

<br>

### Co s chybějícími hodnotami

---

Je důležité vědět, proč máš takové množství chybějících hodnot.

Chybí ti data, protože neexistují, nebo protože nejsou součástí záznamů v datasetu.

V takových krocích je obvykle nutné, pochopit podstatu údajů (dokumentace, specifikace,..).

Nejjednodušší způsoby, jak s chybějícími daty naložit (ale ne efektivní!!):
1. Zahodit chybějící hodnoty,
2. doplnit chybějící hodnoty.

#### Zahodit chybějící hodnoty

In [ ]:
# df_uzivatele.dropna?

In [ ]:
df_uzivatele

In [ ]:
df_uzivatele.dropna()  # axis=0

<br>

V tento okamžik si zahodíš všechny záznamy, které **postrádaly nějakou hodnotu**.

Někdy je jednodušší zahodit sloupeček, který má chybějící hodnotu:

In [ ]:
df_uzivatele.dropna(axis=1)

#### Doplnit automaticky hodnoty

In [ ]:
# df_uzivatele.fillna?

In [ ]:
df_uzivatele

In [ ]:
df_uzivatele.fillna(0)  # replace: "Nan" --> 0

<br>

Pomocí funkce `fillna` můžeš vyplnit místo chybějících hodnot předdefinovanou vlastní hodnotu.

Nebo pomocí argumentu pro `method` pracovat s elegantnějším zadáním.

Nahradit chybějící hodnoty hodnotou, která následuje bezprostředně za ní ve stejném sloupci (To má velký smysl u souborů dat, kde mají pozorování nějaké logické pořadí.)

In [ ]:
df_uzivatele

In [ ]:
df_uzivatele.fillna(method='bfill', axis=0)

In [ ]:
df_uzivatele.fillna(method='ffill', axis=0).fillna(19)

## Encoding

---

Problémy s kódováním jsou běžné.

Jde o proces, který mapuje **bajtové stringy** (`0110101011`) na **uživatelsky čitelný text** (`"ahoj!"`).

Tento problém nastane, pokud se snažíš načíst zdroj v jiné kódovací sadě, než byl soubor zapsaný.

<br>

Protože je těchto sad hodně, občas skončíš s *escapovanými znaky*, nebo s neznámými kliky-háky:

In [ ]:
æ–‡å

<br>

Případně pokud nelze domapovat znaky:

��

<br>

V **Pythonu 2** nebylo lehké *encoding* zajistit.

V **Pythonu 3** je celý proces o dost jednodušší.

Standardem pro práci bývá obyčejně UTF-8. Na to ale zrovna v našich končinách nemůžeš spoléhat.

In [ ]:
veta = "Matouš zaplatil 100 $"

In [ ]:
# veta?

<br>

Údaj, který vidíš výš je datový typ `str`.

In [ ]:
prevedena_veta = veta.encode("utf-8", errors="replace")

In [ ]:
prevedena_veta

In [ ]:
prevedena_veta?

<br>

Dál je možné, převést `str` na `bytes`. Tedy sekvenci čísel.

Jak ale teď tato sekvence vypadá:

In [ ]:
prevedena_veta

<br>

Celou větu můžeš zpátky kódovat pomocí funkce `decode` z `bytes` na `str`:

In [ ]:
print(prevedena_veta.decode("utf-8"))

<br>

Pokud ale zvolíš jinou sadu, nemusíš dostat stejné hodnoty:

In [ ]:
print(prevedena_veta.decode("windows-1250"))

In [ ]:
print(prevedena_veta.decode("ascii"))

<br>

Toto a další neúspěšné postupy je samozřejmě špatně a ty se tomu chceš určitě vyhnout.

In [ ]:
from pandas import read_csv

In [ ]:
# chybna_sada = read_csv("neznamy_vzorek.csv")

<br>

Ne vždy je možné, defaultně pracovat se znakovou sadou **UTF-8**.

Ve výstupu je vidět, že jde o směs několika sad a ty musíš vyzkoušet, jaká bude fungovat.

In [ ]:
import sys

In [ ]:
sys.getdefaultencoding()

In [ ]:
!pip install chardet

In [ ]:
from chardet.universaldetector import UniversalDetector

In [ ]:
detector = UniversalDetector()

In [ ]:
for line in open("neznamy_vzorek.csv", 'rb'):
    detector.feed(line)
    if detector.done:
        break

detector.close()
print(detector.result)

<br>

Pomocí knihovny `chardet` / `charset_normalizer` identifikuješ, o jaké znakové sady jde:

Nyní máš lepší představu o tom, jakou znakovou sadu soubor používá.

Opatrně na délku `str`, který detekuješ.

Pokud je příliš krátký, může to ovlivnit výsledek.

Naopak pokud je příliš dlouhý, může trvat jeho načtení.

In [ ]:
spravna_sada = read_csv("neznamy_vzorek.csv", encoding="Windows-1252")

In [ ]:
spravna_sada.head()

In [ ]:
spravna_sada.info()

<br>

Tentokrát soubor otevřeš pouze s varováním.

Pro příště je určitě výhodou vytvořit kopii takového souboru, kterou uložíš v ideálním kódování:

In [ ]:
spravna_sada.to_csv("kopie_sada_utf8.csv", encoding="utf-8")

In [ ]:
# pokus = read_<zdroj>("vzorek_2.csv")

In [ ]:
# vzorek = to_<zdroj>(...)

<br>

## Nekonzistentní data

---

Pokud ti data nechybí, ještě neznamená, že musí být nutně v pořádku:

In [1]:
from pandas import DataFrame

In [16]:
chybny_dataset = {
    "id": [111, 112, 113, 114, 115, 116, 117],
    "jmeno": ["Matous", "Marek", "Petr", "Filip", "Jan", "Lukas", "David"],
    "vek": [22, 29, 31, 55, 43, 61, 55],
    "zeme": ["Ceska republika", "Slovensko", "Nemecko", "Ceskarepublika", "Ceska Republika", "Rakousko", "Ceska Reapublika"]
}

In [40]:
df_zamestnanci = DataFrame(chybny_dataset)

In [41]:
df_zamestnanci

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceskarepublika
4,115,Jan,43,Ceska Republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska Reapublika


Různé zdroje, správci můžou způsobit nejednotný zápis a dopustit se *nekonzistence*.

In [12]:
df_zamestnanci["zeme"].unique()

array(['Ceska republika', 'Slovensko', 'Nemecko', 'Ceskarepublika',
       'Ceska Republika', 'Rakousko'], dtype=object)

<br>

To můžou být jak malá velká písmena, tak různé znaky, chybějící mezery apod.

Odstranit je není náročné.

Náročné může být opět rozpoznání, takové komplikace.

In [20]:
df_zamestnanci["zeme"].str.lower().unique()

array(['ceska republika', 'slovensko', 'nemecko', 'ceskarepublika',
       'rakousko', 'ceska reapublika'], dtype=object)

<br>

Některé chyby, ale můžou způsobit paseku. Třeba chybějící mezery.

Tady je nejlepší, pomoci si vhodným nástrojem `fuzzywuzzy`.

In [ ]:
!pip install fuzzywuzzy

In [21]:
import fuzzywuzzy
from fuzzywuzzy import process

/home/matous/projects/data-csob-2023/shared/onsite/env/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Pro menší datasety s chybějícími mezerami aj., může tato knihovna pracovat prakticky sama.

Účel této knihovny je rozpoznat podobné řetězce, jaké

In [26]:
shodne = fuzzywuzzy.process.extract(
    "Ceska republika",
    df_zamestnanci["zeme"],
    limit=6,
    scorer=fuzzywuzzy.fuzz.token_sort_ratio
)

In [24]:
df_zamestnanci["zeme"]

0     Ceska republika
1           Slovensko
2             Nemecko
3      Ceskarepublika
4     Ceska Republika
5            Rakousko
6    Ceska Reapublika
Name: zeme, dtype: object

In [28]:
df_zamestnanci["zeme"].unique()

array(['Ceska republika', 'Slovensko', 'Nemecko', 'Ceskarepublika',
       'Ceska Republika', 'Rakousko', 'Ceska Reapublika'], dtype=object)

In [27]:
shodne

[('Ceska republika', 100, 0),
 ('Ceska Republika', 100, 4),
 ('Ceskarepublika', 97, 3),
 ('Ceska Reapublika', 97, 6),
 ('Slovensko', 25, 1),
 ('Nemecko', 18, 2)]

Nyní můžeš vidět shodné `str` se zvoleným zadáním, jak to vidí `fuzzywuzzy`.

Pro nahrazení je optimální nachystat vhodnou uživatelskou funkci:

In [27]:
shodne

[('Ceska republika', 100, 0),
 ('Ceska Republika', 100, 4),
 ('Ceskarepublika', 97, 3),
 ('Ceska Reapublika', 97, 6),
 ('Slovensko', 25, 1),
 ('Nemecko', 18, 2)]

In [29]:
nejblizsi_shoda = [shoda[0] for shoda in shodne if shoda[1] >= 90]

In [30]:
nejblizsi_shoda

['Ceska republika', 'Ceska Republika', 'Ceskarepublika', 'Ceska Reapublika']

In [31]:
shodujici_udaje = df_zamestnanci["zeme"].isin(nejblizsi_shoda)  # membership testing

In [32]:
shodujici_udaje

0     True
1    False
2    False
3     True
4     True
5    False
6     True
Name: zeme, dtype: bool

In [33]:
df_zamestnanci

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceskarepublika
4,115,Jan,43,Ceska Republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska Reapublika


In [39]:
df_zamestnanci

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceska republika
4,115,Jan,43,Ceska republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska republika


In [42]:
df_zamestnanci.loc[shodujici_udaje,"zeme"]

0     Ceska republika
3      Ceskarepublika
4     Ceska Republika
6    Ceska Reapublika
Name: zeme, dtype: object

In [43]:
vysledek = df_zamestnanci.loc[shodujici_udaje, "zeme"] = "Ceska republika"

In [45]:
df_zamestnanci

,id,jmeno,vek,zeme
0,111,Matous,22,Ceska republika
1,112,Marek,29,Slovensko
2,113,Petr,31,Nemecko
3,114,Filip,55,Ceska republika
4,115,Jan,43,Ceska republika
5,116,Lukas,61,Rakousko
6,117,David,55,Ceska republika


In [ ]:
def nahrad_shody_stringem(dframe, jmeno_sloupce, vzor, min_shoda=90):
    
    vsechny_stringy = dframe[sloupec].unique()
    
    shody = fuzzywuzzy.process.extract(vzor, vsechny_stringy, limit=5,
                                       scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    nejblizsi_shoda = [shoda [0] for shoda in shody if shoda[1] >= min_shoda]
    
    shodujici_zaznam = dframe[sloupec].isin(nejblizsi_shoda)
    dframe.loc[shodujici_zaznam, sloupec] = vzor

In [ ]:
nahrad_shody_stringem(dframe=df_zamestnanci, jmeno_sloupce="zeme", vzor="Ceska republika")

In [ ]:
df_zamestnanci

<br>

#### **🧠 CVIČENÍ 🧠, procvič si práci s chybným datasetem**

Analyzuj prodeje produktu ve fiktivní společnosti během jednoho roku a zjisti následující:
1. Nahraj soubor do `DataFrame`, jméno souboru `cviceni_3.csv` (správný encoding, správné rozdělení),
2. projdi sloupce, zkontroluj, kde je problém s daty,
3. vypiš všechny záznamy, které mají ve sloupci `mesto` hodnotu `Praha`.

In [ ]:
from pandas import read_csv

In [ ]:
# ze souboru DFrame (chardet / charset_normalizer)

In [ ]:
cviceni_3_df = read_csv("cviceni_3.csv")  # ??? Doplnit delší zdr. soubor (fce Download)

In [ ]:
# analýza dat v DFrame

In [ ]:
# v 'mesto' --> vzor: 'Praha': 'praaahaaa', 'praha', ..

In [ ]:
import pandas as pd
import re
import fuzzywuzzy
from fuzzywuzzy import process

In [ ]:
data = pd.read_csv(
    'C:\\Users\\je97214\\Desktop\\cviceni_3.csv',
    encoding = 'cp1250',
    sep = ';'
)

In [ ]:
def nahrad_shody_stringem(dframe, sloupec, vzor, min_shoda=60):
    vsechny_stringy = dframe[sloupec].unique()
    shody = fuzzywuzzy.process.extract(vzor, vsechny_stringy, limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    nejblizsi_shoda = [shoda [0] for shoda in shody if shoda[1] >= min_shoda]
    shodujici_zaznam = dframe[sloupec].isin(nejblizsi_shoda)
    dframe.loc[shodujici_zaznam, sloupec] = vzor

In [ ]:
print( 'Problémy s daty:' )
display( data[~data['narozeni'].str.contains('\\d{1,2}/\\d{1,2}/\\d{4}', regex= True, na=False)] )
display( data['mesto'].unique( ) )

In [ ]:
#oprava narození
data['narozeni'] = data['narozeni'].astype( 'datetime64' )
#oprava města
nahrad_shody_stringem(dframe=data, sloupec="mesto", vzor="Praha")
print( 'Výsledek:' )
data.query( "mesto == 'Praha'" )

<details>
    <summary>▶️ Řešení</summary>
    
    ```python
    df_ukazka = read_csv("cviceni_3.csv", encoding="Windows-1250", delimiter=";")

    def nahrad_shody_stringem(df, vzor, vyber, min_shoda=85):
        shody = fuzzywuzzy.process.extract(vzor, vyber, limit=20,
                                           scorer=fuzzywuzzy.fuzz.token_sort_ratio)

        nejblizsi_shoda = [shoda[0] for shoda in shody if shoda[1] >= min_shoda]

        shodujici_zaznam = vyber.isin(nejblizsi_shoda)
        df.loc[shodujici_zaznam, "mesto"] = vzor
        
    nahrad_shody_stringem(df_ukazka, "Praha", df_ukazka["mesto"])
    vystup = df_ukazka[df_ukazka["mesto"] == "Praha"]
    ```
</details>

---